In [1]:
%pip install tensorflow[and-cuda]==2.15.1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!export CUDNN_PATH=$(dirname $(python3 -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))
!export LD_LIBRARY_PATH=${CUDNN_PATH}/lib

!echo $CUDNN_PATH
!ls $CUDNN_PATH


 20180596_Yazan-Matarweh.png   haarcascade_frontalface_default.xml
 emb_den_wei.pkl	       indy2.jpg
 file_modelatm2.h5	       indy.jpg
 file_modelatm2.keras	       TEMP_model_test.html
 file_modelatm3.keras	       TEMP_model_test.ipynb
 file_model.h5		       TEMP_model_test.py
'GradProject[1].ipynb'	       WIN_20240517_22_16_08_Pro.jpg


In [3]:
import tensorrt
import cv2, numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt
import os
from tensorflow.keras.layers import  Conv2D, Dense, MaxPooling2D, Input, Flatten, Layer
# from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model,Model


from time import sleep


2024-05-20 19:52:50.771966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 19:52:50.771994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 19:52:50.772615: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-20 19:52:50.776498: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 19:52:51.414244: W tensorflow/compiler/tf2

In [4]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print(gpu_devices)
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

[]


2024-05-20 19:52:52.053860: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-20 19:52:52.053882: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: yazan-Lenovo-Legion-5-Pro-16ACH6H
2024-05-20 19:52:52.053887: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: yazan-Lenovo-Legion-5-Pro-16ACH6H
2024-05-20 19:52:52.053912: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 550.54.15
2024-05-20 19:52:52.053930: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  550.54.15  Release Build  (dvs-builder@U16-A24-23-2)  Tue Mar  5 22:15:33 UTC 2024
GCC version:  gcc version 11.4.0 

In [5]:
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    ## Magic happens here - similarity calculation
    def call(self, inputs):
        input_embedding, validation_embedding = inputs
        return tf.math.abs(input_embedding - validation_embedding)
    



In [6]:
# model = load_model("file_modelatm3.keras",custom_objects={'L1Dist':L1Dist})
# model.training = False

# for layer in model.layers: 
#     print(layer.get_config(), layer.get_weights())
    
#     # print(layer.get_weights())

# model.summary()

In [7]:
# lays = model.layers

# lays_dict = dict(  [ (lay.name,lay) for lay in lays]     )

# print(lays_dict['input_img'],lays_dict['validation_img'],sep='\n')

# embedding_weights = lays_dict['Embedding_Layer'].get_weights()
# dense_weights, dense_biases = lays_dict['dense_1'].get_weights()

In [8]:
def Embedding_Layer(): 
    input_layer = Input(shape=(100,100,3), name='input_image')
    
    #first block
    convolution_layer1 = Conv2D(64, (10,10), activation='relu')(input_layer)
    max_pooling1 = MaxPooling2D(64, (2,2), padding='same')(convolution_layer1)
    
    #second block
    convolution_layer2 = Conv2D(128, (7,7), activation='relu')(max_pooling1)
    max_pooling2 = MaxPooling2D(64, (2,2), padding='same')(convolution_layer2)
    
    #third block 
    convolution_layer3 = Conv2D(128, (4,4), activation='relu')(max_pooling2)
    max_pooling3 = MaxPooling2D(64, (2,2), padding='same')(convolution_layer3)
    
    #fourth & final block
    convolution_layer4 = Conv2D(256, (4,4), activation='relu')(max_pooling3)
    flatten_layer1 = Flatten()(convolution_layer4)
    dense_layer1 = Dense(4096, activation='sigmoid')(flatten_layer1)
    
    
    return Model(inputs=input_layer, outputs=dense_layer1, name='Embedding_Layer')
    
model1= Embedding_Layer()

def siamese_model(): 
    
    # base image input into the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    siameselayer = L1Dist(name='l1_layer')
    dist = siameselayer([model1(input_image), model1(validation_image)])
    
    # Classification layer 
    classify = Dense(1, activation='sigmoid',name='output_layer')(dist)
    
    return Model(inputs=[input_image, validation_image], outputs=classify, name='SiameseNetwork')


SiameseModel = siamese_model()
# SiameseModel.get_layer('Embedding_Layer').set_weights(embedding_weights)
# SiameseModel.get_layer('output_layer').set_weights((dense_weights,dense_biases))

# for layer in SiameseModel.layers: 
#     print(layer.get_config(), layer.get_weights())


In [9]:
SiameseModel.summary()


Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 Embedding_Layer (Functiona  (None, 4096)                 3896044   ['input_img[0][0]',           
 l)                                                       8          'validation_img[0][0]']      
                                                                                     

In [10]:
# for layer in model.layers: 
#     print(layer.get_config(), layer.get_weights())


In [11]:
print(SiameseModel.get_layer('Embedding_Layer').get_weights() )


[array([[[[-2.16603279e-02,  2.55029798e-02,  2.20325552e-02, ...,
           1.32099204e-02, -2.30212025e-02,  2.86543816e-02],
         [ 1.23531520e-02, -2.24412698e-02,  2.38936022e-02, ...,
          -1.77199207e-03,  2.09151283e-02, -1.86011083e-02],
         [ 2.09788904e-02,  7.24476576e-03, -1.91929080e-02, ...,
           2.08765268e-03, -2.57740580e-02,  8.74548778e-03]],

        [[-1.57967135e-02, -1.73220206e-02,  1.04674883e-02, ...,
          -4.08367068e-03, -2.97546368e-02,  1.37305185e-02],
         [-2.65578870e-02,  4.93709370e-03, -1.81118287e-02, ...,
           2.20267065e-02, -2.80162711e-02, -2.57801432e-02],
         [ 1.65608563e-02, -8.14727694e-03, -7.91210122e-03, ...,
          -2.14564465e-02,  1.49160288e-02,  1.63281858e-02]],

        [[-1.68236382e-02, -2.26514600e-03,  1.03259422e-02, ...,
          -2.11549364e-03, -1.40387546e-02, -2.84577906e-02],
         [ 2.24030577e-02, -1.66119486e-02,  7.41917640e-03, ...,
          -2.99229752e-02,  1.083

In [12]:
import pickle
embedding_weights, dense_weights = pickle.load(open('emb_den_wei.pkl','rb'))
SiameseModel.get_layer('Embedding_Layer').set_weights(embedding_weights)
SiameseModel.get_layer('output_layer').set_weights(dense_weights)

In [13]:
# SiameseModel.save(filepath='./file_modelatm3.keras')

In [14]:
model = load_model("file_modelatm3.keras",custom_objects={'L1Dist':L1Dist})

In [15]:
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 Embedding_Layer (Functiona  (None, 4096)                 3896044   ['input_img[0][0]',           
 l)                                                       8          'validation_img[0][0]']      
                                                                                     

In [21]:
def preprocessing_func(file_path):
    
    # Read in image from file path
    byte_image = tf.io.read_file(file_path)
    # Load image 
    imag = tf.io.decode_jpeg(byte_image)
    
    # Preprocessing: resize image -> 100*100*3
    imag = tf.image.resize(imag, (100,100))
    # Scale image to be between 0 and 1 
    imag = imag / 255.0
    imag = np.expand_dims(imag,axis=0)
    return imag


# def preprocess_identical(input_img, validation_img, label):
#     #label: indicates whether the input and validation images are identical or not
#     return(preprocessing_func(input_img), preprocessing_func(validation_img), label)

def get_face(file_path):
    # byte_image = tf.io.read_file(file_path)
    # img = tf.io.decode_jpeg(byte_image) if '.jpg' in file_path or '.jpeg' in file_path else tf.io.decode_png(byte_image)

    img = cv2.imread(file_path)
    face_classifier = cv2.CascadeClassifier(os.path.join(cv2.data.haarcascades, "haarcascade_frontalface_default.xml"))
    
    classifier = face_classifier.detectMultiScale( 
    img, scaleFactor=1.1, minNeighbors=9) 

    x0,y0,w,h = classifier[0]
    
    x1 = x0 + w
    y1 = y0 + h
    face = img[y0:y1,x0:x1]
    face = face / 255.0
    face = tf.image.resize(face,(100,100))
    return np.expand_dims(face, axis=0)


    

In [22]:
filename_known1 = '../Known_Faces/20180596_Yazan-Matarweh.png'
# filename_known1 = '20180596_Yazan-Matarweh.png'
filename_unknown1 = "../../../../../Pictures/Camera Roll/WIN_20240517_22_16_08_Pro.jpg"

filename_known2 = '../Known_Faces/20021736_Anne-Hathaway.png'

filename_known3 = '../Known_Faces/19850192_Rosie-O\'Donnell.jpg'

known1 = get_face(filename_known1)

unknown1 = get_face(filename_unknown1)

known2 = get_face(filename_known2)

known3 = get_face(filename_known3)
# label = filename_known.split('/')[-1].split('_')[1]

challenges =  ([known1,unknown1],[known2,unknown1],[known3,unknown1])

for challenge in challenges:
    prediction = SiameseModel.predict(challenge)
    print(prediction)



a
a
a
1/1 [==============================] - 0s 71ms/step
[[0.50757736]]
1/1 [==============================] - 0s 79ms/step
[[0.9998095]]
1/1 [==============================] - 0s 82ms/step
[[0.99892604]]
